In [1]:
import re
import json
from itertools import chain

from datetime import datetime
from datetime import date, timedelta, time

import pandas as pd
from pandas import DataFrame, Index
from tqdm.notebook import tqdm

from usage_events_processing import get_model_dataframe
from utils import UUID2ID_DICT, UUIDS, IDS, getPkgName

Reference: https://developer.android.com/reference/android/app/usage/UsageEvents.Event

#### 0: "NONE"
- No event type.
#### 1: "ACTIVITY_RESUMED"
- An event type denoting that an Activity moved to the foreground. 
- This event has a package name and class name associated with it and can be retrieved using getPackageName() and getClassName().
- If a package has multiple activities, this event is reported for each activity that moves to foreground
- This event is corresponding to `Activity.onResume()` of the activity's lifecycle.
#### 2: "ACTIVITY_PAUSED"
- An activity becomes invisible on the UI, corresponding to Activity.onStop() of the activity's lifecycle.
#### 5: "CONFIGURATION_CHANGE"
- An event type denoting that the device configuration has changed.
#### 7: "USER_INTERACTION"
- An event type denoting that a package was interacted with in some way by the user.
#### 8: "SHORTCUT_INVOCATION"
- An event type denoting that an action equivalent to a ShortcutInfo is taken by the user.
#### 11: "STANDBY_BUCKET_CHANGED"
- An event type denoting a change in App Standby Bucket. The new bucket can be retrieved by calling getAppStandbyBucket().
#### 15: "SCREEN_INTERACTIVE"
- An event type denoting that the screen has gone in to an interactive state (turned on for full user interaction, not ambient display or other non-interactive state).
#### 16: "SCREEN_NON_INTERACTIVE"
- An event type denoting that the screen has gone in to a non-interactive state (completely turned off or turned on only in a non-interactive state like ambient display).
#### 17: "KEYGUARD_SHOWN"
- An event type denoting that the screen's keyguard has been shown, whether or not the screen is off.
#### 18: "KEYGUARD_HIDDEN"
- An event type denoting that the screen's keyguard has been hidden.
- This typically happens when the user unlocks their phone after turning it on.
#### 19: "FOREGROUND_SERVICE_START"
- An event type denoting start of a foreground service.
- This event has a package name and class name associated with it and can be retrieved using getPackageName() and getClassName(). 
- If a package has multiple foreground services, this event is reported for each service that is started.
#### 20: "FOREGROUND_SERVICE_STOP"
- An event type denoting stop of a foreground service.
- This event has a package name and class name associated with it and can be retrieved using getPackageName() and getClassName().
- If a package has multiple foreground services, this event is reported for each service that is stopped.
#### 23: "ACTIVITY_STOPPED"
- An activity becomes invisible on the UI, corresponding to Activity.onStop() of the activity's lifecycle.
#### 26: "DEVICE_SHUTDOWN"
- An event type denoting that the Android runtime underwent a shutdown process.
#### 27: "DEVICE_STARTUP"
- An event type denoting that the Android runtime started up.

In [2]:
df_ = get_model_dataframe("UsageEvents")
df_ = df_.rename(columns={"packageName": "pkgName"})

year 537780 is out of range
{'eventType': 7, 'packageName': 'com.google.android.apps.messaging', 'time': 1690900046869, 'timeStamp': 16908530853537652}


In [3]:
df = df_

In [4]:
drg_apps = ['com.instagram.android', 'com.zhiliaoapp.musically', 'com.snapchat.android', 'com.example.drg2023']

df = df[df['pkgName'].isin(drg_apps)]

app_mapping = {
    'com.instagram.android': 'Instagram',
    'com.zhiliaoapp.musically': 'TikTok',
    'com.snapchat.android': 'Snapchat',
    'com.example.drg2023': 'AppMinder'
}

df['appName'] = df['pkgName'].map(app_mapping)
df.drop(columns=['pkgName'], inplace=True)

df = df.sort_values(by=['id', 'date'], axis=0)

/var/folders/vr/fynfqqj539zdh2hh2gwbpjn40000gp/T/ipykernel_85819/1141506027.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['appName'] = df['pkgName'].map(app_mapping)
/var/folders/vr/fynfqqj539zdh2hh2gwbpjn40000gp/T/ipykernel_85819/1141506027.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['pkgName'], inplace=True)


In [5]:
df.columns

Index(['id', 'uuid', 'datetime', 'eventType', 'time', 'date', 'className',
       'configuration', 'shortcutId', 'timeStamp', 'appName'],
      dtype='object')

In [6]:
df.sample(10)

,id,uuid,datetime,eventType,time,date,className,configuration,shortcutId,timeStamp,appName
42284,p008,014b6780-c3bc-45a6-9937-096bd0b79b62,2023-07-17 16:50:25,ACTIVITY_STOPPED,1.689706e+12,2023-07-18,com.example.drg2023.MainActivity,NaN,NaN,NaN,AppMinder
132682,p107,9c8a2381-f3fe-460e-ada4-2f2b3a5c24c6,2023-07-25 10:30:51,ACTIVITY_STOPPED,1.690356e+12,2023-07-26,com.snap.mushroom.MainActivity,NaN,NaN,NaN,Snapchat
195156,p062,640b0635-3307-4326-ac1e-8606b117be01,2023-08-02 02:21:56,ACTIVITY_RESUMED,1.691044e+12,2023-08-02,com.ss.android.ugc.aweme.main.MainActivity,NaN,NaN,NaN,TikTok
1433,p062,640b0635-3307-4326-ac1e-8606b117be01,2023-07-28 13:45:10,NOT_DEFINED,1.690631e+12,2023-07-29,NaN,NaN,NaN,NaN,Snapchat
100974,p129,d24f3da7-0363-4cea-983b-b0160c7f3f81,2023-08-15 11:14:27,ACTIVITY_STOPPED,1.692167e+12,2023-08-16,com.ss.android.ugc.aweme.main.MainActivity,NaN,NaN,NaN,TikTok
29555,p187,0d30bc09-dc43-4eee-a427-34b3ca6041e6,2023-07-30 23:02:32,ACTIVITY_STOPPED,1.690857e+12,2023-07-31,com.ss.android.ugc.aweme.host.TikTokHostActivity,NaN,NaN,NaN,TikTok
264659,p172,f215e668-a529-4726-9cd3-ec12b807bb55,2023-08-08 21:43:13,ACTIVITY_RESUMED,1.691577e+12,2023-08-09,com.instagram.mainactivity.MainActivity,NaN,NaN,NaN,Instagram
11554,p123,e7d22ac2-5ef8-4ef8-8755-1b88a93854a2,2023-08-04 11:43:21,ACTIVITY_PAUSED,1.691209e+12,2023-08-04,com.example.drg2023.MainActivity,NaN,NaN,NaN,AppMinder
32498,p096,24cc3fc0-40f1-4acd-9074-a32f1ab0db3f,2023-07-31 18:53:05,NOT_DEFINED,1.690866e+12,2023-07-31,NaN,NaN,NaN,NaN,Instagram
85752,p194,db2ac786-0a6d-4f1b-a8b2-034c8311f0fc,2023-08-09 09:25:53,ACTIVITY_RESUMED,1.691626e+12,2023-08-09,com.ss.android.ugc.aweme.detail.ui.DetailActivity,NaN,NaN,NaN,TikTok


In [7]:
df.drop(columns=['configuration', 'shortcutId', 'timeStamp'], inplace=True)

In [8]:
len(df)

1651505

In [9]:
df.to_csv('./usage_events_processed.csv')

In [10]:
df

,id,uuid,datetime,eventType,time,date,className,appName
156701,p008,014b6780-c3bc-45a6-9937-096bd0b79b62,2023-07-16 16:02:26,ACTIVITY_RESUMED,1.689638e+12,2023-07-17,com.instagram.modal.ModalActivity,Instagram
160488,p008,014b6780-c3bc-45a6-9937-096bd0b79b62,2023-07-16 16:02:26,ACTIVITY_RESUMED,1.689638e+12,2023-07-17,com.instagram.modal.ModalActivity,Instagram
156713,p008,014b6780-c3bc-45a6-9937-096bd0b79b62,2023-07-16 16:04:28,ACTIVITY_PAUSED,1.689638e+12,2023-07-17,com.instagram.modal.ModalActivity,Instagram
156714,p008,014b6780-c3bc-45a6-9937-096bd0b79b62,2023-07-16 16:04:28,ACTIVITY_STOPPED,1.689638e+12,2023-07-17,com.instagram.modal.ModalActivity,Instagram
160500,p008,014b6780-c3bc-45a6-9937-096bd0b79b62,2023-07-16 16:04:28,ACTIVITY_PAUSED,1.689638e+12,2023-07-17,com.instagram.modal.ModalActivity,Instagram
...,...,...,...,...,...,...,...,...
99706,p208,48c16a98-819b-40f7-891a-ea8830b3bfae,2023-09-01 13:51:16,ACTIVITY_RESUMED,1.693602e+12,2023-09-01,com.instagram.mainactivity.MainActivity,Instagram
99707,p208,48c16a98-819b-40f7-891a-ea8830b3bfae,2023-09-01 13:51:16,ACTIVITY_STOPPED,1.693602e+12,2023-09-01,com.instagram.modal.TransparentModalActivity,Instagram
99714,p208,48c16a98-819b-40f7-891a-ea8830b3bfae,2023-09-01 13:56:47,ACTIVITY_PAUSED,1.693602e+12,2023-09-01,com.instagram.mainactivity.MainActivity,Instagram
99716,p208,48c16a98-819b-40f7-891a-ea8830b3bfae,2023-09-01 13:56:48,ACTIVITY_STOPPED,1.693602e+12,2023-09-01,com.instagram.mainactivity.MainActivity,Instagram


In [11]:
# rate of non-response?
# did they spend less time on Instagram?